In [1]:
import os
import asyncio
import threading
import time
import base64
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional
import sqlite3
from contextlib import asynccontextmanager
import json
from fastapi.responses import FileResponse


from fastapi import FastAPI, HTTPException, BackgroundTasks, Depends, File, UploadFile, Form
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field, validator
from dotenv import load_dotenv
import uvicorn
from playsound import playsound
import io

# LangGraph and LangChain imports
from langgraph.graph import StateGraph, END, START
from langchain_core.messages import HumanMessage
from langchain_groq import ChatGroq
from fastapi.staticfiles import StaticFiles 
# Load environment variables

In [1]:
row={
  'id': 1,
  'name': 'Alice',
  'age': 30
}


In [ ]:
row['id']#Dictionary behaviour

1

In [4]:
row['0']

KeyError: '0'

In [5]:
row=(1, 'Alice', 30)  # A plain tuple


In [8]:
row[0]

1

In [14]:
response = [{
    "success": "true/false",
    "reasoning": "detailed explanation of what you see and why it passes/fails verification",
    "confidence": 0.0
}]


In [17]:
response[0]

{'success': 'true/false',
 'reasoning': 'detailed explanation of what you see and why it passes/fails verification',
 'confidence': 0.0}

In [18]:
response = {
    "success": "true/false",
    "reasoning": "detailed explanation of what you see and why it passes/fails verification",
    "confidence": 0.0
}


In [20]:
for key,value in response.items():
    print(key,":",value)

success : true/false
reasoning : detailed explanation of what you see and why it passes/fails verification
confidence : 0.0


AttributeError: 'dict' object has no attribute 'content'

  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_text_splitters-0.3.9-py3-none-any.whl.metadata (1.9 kB)
  Using cached sqlalchemy-2.0.43-cp313-cp313-win_amd64.whl.metadata (9.8 kB)
Using cached langchain-0.3.27-py3-none-any.whl (1.0 MB)
Using cached langchain_text_splitters-0.3.9-py3-none-any.whl (33 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ----------------------------- ---------- 1.6/2.1 MB 10.4 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 9.9 MB/s eta 0:00:00

   ---------------------------------------- 0/4 [greenlet]
   ---------------------------------------- 0/4 [greenlet]
   ---------------------------------------- 0/4 [greenlet]
   ---------- ----------------------------- 1/4 [SQLAlchemy]
   ---------- ----------------------------- 1/4 [SQLAlchemy]
   ---------- ----------------------------- 1/4 [SQLAlchemy]
   ---------- ----------------------------- 1/4 [SQLAlchemy]
   ---


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [27]:
pip install langchain_community

  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached aiohttp-3.12.15-cp313-cp313-win_amd64.whl.metadata (7.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached numpy-2.3.2-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.7.0-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached multidict-6.6.4-cp313-cp313-win_amd64.whl.metadata (5.4 kB)
  Using cached propcache-0.3.2-cp313-cp313-win_amd64.whl.metadata (12 kB)
  Using cached yarl-1.20.1-cp313-cp313-win_amd64.whl.metadata (76 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
 


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import os
from dotenv import load_dotenv
from groq import Groq
from langchain.chat_models import ChatGroq

load_dotenv()  # Loads GROQ_API_KEY from .env file if present

# Initialize Groq client
groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# Initialize LangChain ChatGroq model
chat = ChatGroq(client=groq_client)


ImportError: cannot import name 'ChatGroq' from 'langchain.chat_models' (d:\CA content\Python\Projects\Sch\venv\Lib\site-packages\langchain\chat_models\__init__.py)

In [2]:
class WorkflowState(BaseModel):
    task_id: int
    current_time: str
    task_start_time: str
    verification_instructions: str
    image_data: Optional[str] = None
    verification_result: Optional[Dict[str, Any]] = None
    step: str = "check_time"
    alarm_active: bool = False

In [3]:
active_tasks:dict[int,WorkflowState]

In [4]:
# First, create a WorkflowState instance
task_state = WorkflowState(
    task_id=123,  # example task ID
    current_time="10:00",  # example current time
    task_start_time="09:30",  # example start time
    verification_instructions="Check all systems",  # example instructions
    # image_data and verification_result are optional, so you can omit them
    step="check_time",  # default value
    alarm_active=False  # default value
)


In [5]:
# Then add it to active_tasks
active_tasks = {}  # initialize if not already initialized
active_tasks[task_state.task_id] = task_state

In [6]:
active_tasks

{123: WorkflowState(task_id=123, current_time='10:00', task_start_time='09:30', verification_instructions='Check all systems', image_data=None, verification_result=None, step='check_time', alarm_active=False)}

In [7]:
active_tasks = {
    123: WorkflowState(
        task_id=123,
        current_time="10:00",
        task_start_time="09:30",
        verification_instructions="Check system A",
        step="verify_data"
    ),
    456: WorkflowState(
        task_id=456,
        current_time="11:15",
        task_start_time="10:45",
        verification_instructions="Inspect component B",
        image_data="base64_encoded_image_data",
        step="review_image"
    )
}

In [ ]:
"content": "{\n \"success\": true,\n \"reasoning\": \"The image shows a person holding up a book or notebook. The object being held appears to be a spiral-bound book with a visible cover, meeting the requirement of holding a book. The image is somewhat blurry but still clear enough to verify the requirements. The person is visible from the chest up, and their face is recognizable. The background appears to be an indoor setting.\",\n \"confidence\": 0.8\n}",

SyntaxError: invalid syntax (3343146491.py, line 1)

In [ ]:
import re
raw_content=.strip()
if raw_content.startswith("```"):
      raw_content = re.sub(r"^```(?:json)?\s*|\s*```$", "", raw_content, flags=re.DOTALL).strip()
                  